# ![FUNWAVE Banner](media/funwave_banner.png)

This Notebook runs simple one-dimensional FUNWAVE simulations. 
-----

It consists of the following four steps:

* **Step #1** - Bathymetry: In this step you can generate the one-dimensional bathymetry that will be used in the FUNWAVE simulation. The user has the option of creating a flat bathymetry, a slope bathymetry or upload his own file. Once the depth file is generated, proceed to Step #2.

* **Step #2** - Input File: In this tab you can generate the Input file with your desired specifications for the simulation. After generating your input file, proceed to Step #3.

* **Step #3** - Run Funwave: This tab lets you run FUNWAVE without the need of a terminal. You can also see the simulation's proggress and abort it if it is necessary. Proceed to Step #4 to process your results.

* **Step #4** - Post Processing: This tab gives you the option of generating an image at an specific time or generate a video of the simulation.

In [1]:
import pyFiles.main1D

VBox(children=(Box(layout=Layout(height='20px', width='90%')), HBox(children=(HTML(value='Project Title:'), Text(value='meh4', layout=Layout(width='40%')), Box(layout=Layout(width='10%')), Button(description='Generate Project', layout=Layout(height='40px', width='30%'), style=ButtonStyle())), layout=Layout(width='90%')), Box(layout=Layout(height='20px', width='90%')), Tab(children=(VBox(children=(VBox(children=(HTML(value='Specify the type of One-Dimensional Bathymetry:', layout=Layout(height='25px')), Dropdown(options=('Select Bathy', 'Upload File', 'Slope', 'Flat'), value='Select Bathy')), layout=Layout(height='70px')), VBox(children=(VBox(children=(HTML(value='<ul> \n<li><b>Substep 1:</b> Upload your file in the <b>Project Title</b> folder through  the <b>Jupyter Notebook home directory</b>. Go inside your Projet\'s folder and upload the file by pressing the "Upload" button located at the top right corner of the directory.<br></li>\n\n<b>NOTE: </b>This file must be a text of 1 row; with depth values [-] for underwater and [+] for surface. Also, the values must be in <b>metric</b> units.<br><br>\n\n<li><b>Substep 2:</b> Once the file is uploaded, identify its name (e.g. MyBathy.txt) and its total horizontal length (THL) in the widgets below. Press "Plot Bathymetry" to visualize the bathymetry.<br><br></li>\n\n<li><b>Substep 3:</b> If you are satisfied with the bathymetry, <b>you must press "Assemble Bathymetry File"</b> before continuing to <b>Step #2</b>. This will format the uploaded bathymetry file to the <b>required FUNWAVE format.</b><br><br></li>\n</ul>\n', layout=Layout(width='90%')), Box(layout=Layout(height='60px', width='5%')), HBox(children=(Box(layout=Layout(height='60px', width='5%')), HTML(value='File name:', layout=Layout(width='10%')), Text(value='', layout=Layout(width='25%')), BoundedFloatText(value=200.0, description='THL', layout=Layout(width='25%'), max=1500.0, min=10.0, step=0.01)), layout=Layout(height='75px'))), layout=Layout(display='none', height='500px')), VBox(children=(HTML(value='This Option plots and saves simple one-dimensional <b>slope</b>\nbathymetries consisting of 2 segments and 3 vertices. The user can select the <b>total horizontal lenght</b> \nof the bathymetry <b>(THL)</b>, the <b>spacing</b> between the points <b>(DX)</b>,\nand the <b>elevation</b> and <b>location</b> of the vertices. Once the desired bathymetry is plotted,\npress the "Assemble Bathymetry File"\nbutton and proceed to Step #2.<br>The values are in <b>metric</b> units.', layout=Layout(width='90%')), Box(layout=Layout(height='25px')), HBox(children=(BoundedFloatText(value=200.0, description='THL', layout=Layout(width='25%'), max=1500.0, min=10.0, step=0.01), BoundedFloatText(value=1.0, description='DX', layout=Layout(width='25%'), step=0.5)), layout=Layout(align_items='center', height='40px', width='90%')), Box(layout=Layout(height='25px')), HTML(value='<b>NOTE:</b> Depth values are [-] for underwater and [+] for surface.'), HBox(children=(VBox(children=(HTML(value='<b>Vertex #1<b/>'), FloatSlider(value=-10.0, description='Elevation ', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), HTML(value='Located at <b>0.0 meters</b>.')), layout=Layout(align_items='center', border='solid 2px grey', display='flex-grow', width='30%')), VBox(children=(HTML(value='<b>Vertex #2<b/>'), FloatSlider(value=-10.0, description='Elevation ', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), FloatSlider(value=10.0, description='Location', layout=Layout(height='50px', width='100%'), max=200.0, step=0.01)), layout=Layout(align_items='center', border='solid 2px grey', display='flex-grow', width='30%')), VBox(children=(HTML(value='<b>Vertex #3<b/>'), FloatSlider(value=-10.0, description='Elevation', layout=Layout(height='50px', width='100%'), max=15.0, min=-10.0, step=0.01), HTML(value='Located at <b>THL</b> distance.')), layout=Layout(align_items='center', border='solid 2px grey', display='fl